In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

- id : 데이터 고유 id
- OverallQual : 전반적 재료와 마감 품질
- YearBuilt : 완공 연도
- YearRemodAdd : 리모델링 연도
- ExterQual : 외관 재료 품질
- BsmtQual : 지하실 높이
- TotalBsmtSF : 지하실 면적 
- 1stFlrSF : 1층 면적 
- GrLivArea : 지상층 생활 면적
- FullBath : 지상층 화장실 개수 
- KitchenQual : 부억 품질 
- GarageYrBlt : 차고 완공 연도
- GarageCars: 차고 자리 개수
- GarageArea: 차고 면적 
- target : 집값(달러 단위)

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [3]:
train = train.iloc[:, 1:]
test = test.iloc[:, 1:]

In [4]:
train = train[train['Garage Yr Blt'] < 2050]

In [5]:
train['new1'] = 2022 - train['Year Remod/Add']
test['new1'] = 2022 - test['Year Remod/Add']

In [6]:
train['new2'] = np.log1p(train['1st Flr SF'] / train['Total Bsmt SF'])
test['new2'] = np.log1p(test['1st Flr SF'] / test['Total Bsmt SF'])

In [7]:
train['new3'] = train['Gr Liv Area'] / train['1st Flr SF']
test['new3'] = test['Gr Liv Area'] / test['1st Flr SF']

In [8]:
train['EQ'] = train['Exter Qual'].map({'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1})
test['EQ'] = test['Exter Qual'].map({'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1})

In [9]:
train['KQ'] = train['Kitchen Qual'].map({'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1})
test['KQ'] = test['Kitchen Qual'].map({'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1})

In [10]:
train['BQ'] = train['Bsmt Qual'].map({'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1})
test['BQ'] = test['Bsmt Qual'].map({'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1})

In [11]:
train['TQ'] = train['Overall Qual'] * 0.5 + train['EQ'] * 0.3 + train['KQ'] * 0.15 + train['BQ'] * 0.05
test['TQ'] = test['Overall Qual'] * 0.5 + test['EQ'] * 0.3 + test['KQ'] * 0.15 + test['BQ'] * 0.05

In [12]:
cat_cols = ['Exter Qual', 'Kitchen Qual', 'Bsmt Qual']

In [13]:
for c in cat_cols :
    ord_df = train.groupby(c).target.median().reset_index(name = f'ord_{c}')
    train = pd.merge(train, ord_df, how = 'left')
    test = pd.merge(test, ord_df, how = 'left')

In [14]:
train.drop(cat_cols, axis = 1, inplace = True)
test.drop(cat_cols, axis = 1, inplace = True)

In [15]:
X = train.drop(['KQ', 'EQ', 'BQ', 'Overall Qual', 'Garage Area', '1st Flr SF', 'target'], axis = 1)
y = np.log1p(train.target)

target = test[X.columns]

In [16]:
target.fillna(target.median(), inplace = True)

In [17]:
kf = KFold(n_splits = 10, random_state = 42, shuffle = True)

In [18]:
n_models = 6

In [20]:
def NMAE(true, pred) -> float:
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

In [21]:
val_nmae = []
rf_nmae = []
ngb_nmae = []
xgb_nmae = []
cb_nmae = []
gbr_nmae = []
sr_nmae = []
fold_pred = np.zeros(target.shape[0])
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'##### {n + 1} FOLD Training..... #####')
    
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    tr_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    target_data = Pool(data = target, label = None)
    
    rf = RandomForestRegressor(random_state = 2022, criterion = 'mae')
    cb = CatBoostRegressor(depth = 4, random_state = 2, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)
    ngb = NGBRegressor(random_state = 1, n_estimators = 3000, verbose = 0, learning_rate = 0.03)
    xgb = XGBRegressor(random_state = 28, max_depth = 4, n_estimators = 3000, learning_rate = 0.03, objective = 'reg:squarederror')
    gbr = GradientBoostingRegressor(random_state = 67, max_depth = 4, learning_rate = 0.03, n_estimators = 1500)
    lr = Ridge(random_state = 42)
    
    
    rf.fit(tr_x, tr_y)
    rf_val = np.expm1(rf.predict(val_x))
    rf_score = NMAE(val_y, rf_val)
    rf_nmae.append(rf_score)
    print(f'{rf.__class__.__name__} Training Completed...NMAE = {NMAE(val_y, rf_val)}')
    gbr.fit(tr_x, tr_y)
    gbr_val = np.expm1(gbr.predict(val_x))
    gbr_score = NMAE(val_y, gbr_val)
    gbr_nmae.append(gbr_score)
    print(f'{gbr.__class__.__name__} Training Completed...NMAE = {NMAE(val_y, gbr_val)}')
    cb.fit(tr_data, eval_set = val_data, early_stopping_rounds = 750, verbose = 0)
    cb_val = np.expm1(cb.predict(val_data))
    cb_score = NMAE(val_y, cb_val)
    cb_nmae.append(cb_score)
    print(f'{cb.__class__.__name__} Training Completed...NMAE = {NMAE(val_y, cb_val)}')
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 500)
    ngb_val = np.expm1(ngb.predict(val_x))
    ngb_score = NMAE(val_y, ngb_val)
    ngb_nmae.append(ngb_score)
    print(f'{ngb.__class__.__name__} Training Completed...NMAE = {NMAE(val_y, ngb_val)}')
    xgb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 500, verbose = 0, eval_metric = 'mae')
    xgb_val = np.expm1(xgb.predict(val_x))
    xgb_score = NMAE(val_y, xgb_val)
    xgb_nmae.append(xgb_score)
    print(f'{xgb.__class__.__name__} Training Completed...NMAE = {NMAE(val_y, xgb_val)}')
    sr = StackingRegressor(estimators = [('RF', rf), ('CB', cb), ('XGB', xgb), ('GBR', gbr)], final_estimator = lr)
    sr.fit(tr_x, tr_y)
    sr_val = np.expm1(sr.predict(val_x))
    sr_score = NMAE(val_y, sr_val)
    sr_nmae.append(sr_score)
    print(f'{sr.__class__.__name__} Training Completed...NMAE = {sr_score}\n')
    val_pred = (sr_val + rf_val + ngb_val + cb_val + xgb_val + gbr_val) / n_models
    
    fold_nmae = NMAE(val_y, val_pred)
    val_nmae.append(fold_nmae)
    print(f'{n + 1} FOLD Validation NMAE = {fold_nmae}\n')
    
    rf_fold = rf.predict(target) / (n_models * kf.n_splits)
    cb_fold = cb.predict(target_data) / (n_models * kf.n_splits)
    ngb_fold = ngb.predict(target) / (n_models * kf.n_splits)
    xgb_fold = xgb.predict(target) / (n_models * kf.n_splits)
    gbr_fold = gbr.predict(target) / (n_models * kf.n_splits)
    sr_fold = sr.predict(target) / (n_models * kf.n_splits)
    
    fold_pred += (sr_fold + gbr_fold + rf_fold  + cb_fold + ngb_fold + xgb_fold)
    
print(f'{"#" * 30}')
print(f'{kf.n_splits}FOLD Mean of NMAE = {np.mean(val_nmae)} & std = {np.std(val_nmae)}')
print(f'##### 각 모델별 NMAE #####')
print(f'{rf.__class__.__name__} {kf.n_splits} FOLD Mean of NMAE = {np.mean(rf_nmae)}')
print(f'{cb.__class__.__name__} {kf.n_splits} FOLD Mean of NMAE = {np.mean(cb_nmae)}')
print(f'{ngb.__class__.__name__} {kf.n_splits} FOLD Mean of NMAE = {np.mean(ngb_nmae)}')
print(f'{gbr.__class__.__name__} {kf.n_splits} FOLD Mean of NMAE = {np.mean(gbr_nmae)}')
print(f'{xgb.__class__.__name__} {kf.n_splits} FOLD Mean of NMAE = {np.mean(xgb_nmae)}')
print(f'{sr.__class__.__name__} {kf.n_splits} FOLD Mean of NMAE = {np.mean(sr_nmae)}')

##### 1 FOLD Training..... #####
RandomForestRegressor Training Completed...NMAE = 0.08597747959431033
GradientBoostingRegressor Training Completed...NMAE = 0.0816727872223358
CatBoostRegressor Training Completed...NMAE = 0.08729140831662417
NGBRegressor Training Completed...NMAE = 0.08775494370746868
XGBRegressor Training Completed...NMAE = 0.092200131779733
StackingRegressor Training Completed...NMAE = 0.08175945534725439

1 FOLD Validation NMAE = 0.08259599407383793

##### 2 FOLD Training..... #####
RandomForestRegressor Training Completed...NMAE = 0.10898335612979533
GradientBoostingRegressor Training Completed...NMAE = 0.10155448019532137
CatBoostRegressor Training Completed...NMAE = 0.1030231327287978
NGBRegressor Training Completed...NMAE = 0.10286248554957939
XGBRegressor Training Completed...NMAE = 0.11461251601968098
StackingRegressor Training Completed...NMAE = 0.10119270435728361

2 FOLD Validation NMAE = 0.10275528766734192

##### 3 FOLD Training..... #####
RandomForestReg

In [22]:
final = (sr_fold + rf_fold + xgb_fold + ngb_fold + cb_fold + gbr_fold) / n_models

In [23]:
submission['target'] = np.expm1(fold_pred)

In [26]:
submission.to_csv('202.csv', index = False)

In [27]:
submission

,id,target
0,1,322996.157582
1,2,128565.909910
2,3,178316.260011
3,4,240958.460065
4,5,132820.402995
...,...,...
1345,1346,331974.714605
1346,1347,127277.111933
1347,1348,72412.644100
1348,1349,184746.020763
